<a href="https://colab.research.google.com/github/Onethybeing/GANs/blob/main/vaillaGANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
torch.cuda.is_available()

True

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

In [4]:
latent_dim = 100
batch_size = 64
lr = 0.0002
epochs = 50
device = "cuda" if torch.cuda.is_available() else "cpu"

###Generator

In [8]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator,self).__init__()
    self.model = nn.Sequential(
        nn.Linear(latent_dim,256),
        nn.LeakyReLU(0.2,inplace=True),
        nn.Linear(256,512),
        nn.LeakyReLU(0.2,inplace= True),
        nn.Linear(512,1024),
        nn.LeakyReLU(0.2,inplace=True),
        nn.Linear(1024,28*28),
        nn.Tanh()
    )
  def forward(self,x):
    return self.model(x).view(-1,1,28,28)

###Discriminator

In [9]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    self.model = nn.Sequential(
        nn.Linear(28*28,512),
        nn.LeakyReLU(0.2,inplace= True),
        nn.Linear(512,256),
        nn.LeakyReLU(0.2,inplace=True),
        nn.Linear(256,1),
        nn.Sigmoid()
    )
  def forward(self,x):
    return self.model(x.view(-1,28*28))

generator = Generator().to(device)
discriminator = Discriminator().to(device)

In [12]:
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(),lr=lr,betas=(0.5,0.999))
optimizer_D = optim.Adam(discriminator.parameters(),lr = lr,betas=(0.5,0.999))
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])
dataloader = DataLoader(datasets.MNIST("./daat",train = True,download=True,transform =transform),batch_size = batch_size,shuffle = True)
for epochs in range(epochs):
  for i,(imgs, _) in enumerate(dataloader):
    real = imgs.to(device)
    batch_size = real.size(0)

    #labels
    real_labels = torch.ones(batch_size,1).to(device)
    fake_labels = torch.zeros(batch_size,1).to(device)

    #train DIscriminator
    x = torch.randn(batch_size,latent_dim).to(device)
    fake = generator(x)
    real_loss = criterion(discriminator(real),real_labels)
    fake_loss = criterion(discriminator(fake.detach()),fake_labels)
    d_loss = real_loss + fake_loss
    optimizer_D.zero_grad()
    d_loss.backward()
    optimizer_D.step()

    #train generator
    fake_loss = criterion(discriminator(fake),real_labels)
    optimizer_G.zero_grad()
    fake_loss.bacward()
    optimizer_G.step()

  print(f"Epochs {epochs+1}/{epochs} | D Loss : {d_loss.item():.4f}| G Loss: {fake_loss.item():.4f}")
